In [3]:
using PyPlot
import import_ipynb
import A

ArgumentError: ArgumentError: Package import_ipynb not found in current path:
- Run `import Pkg; Pkg.add("import_ipynb")` to install the import_ipynb package.


In [ ]:
function log_studentt(x::Array{Float64,2}, alpha::Float64, sigma::Float64)




    return value::Float64, grad::Array{Float64,2}
end

In [ ]:
# Evaluate stereo log prior.
# Set: alpha=1.0, sigma=1.0
function stereo_log_prior(x::Array{Float64,2})




    return  value::Float64, grad::Array{Float64,2}
end


In [ ]:
# Evaluate stereo log likelihood.
# Set: Alpha = 1.0, Sigma = 0.004
function stereo_log_likelihood(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return value::Float64, grad::Array{Float64,2}
end

In [ ]:
# Evaluate stereo posterior
function stereo_log_posterior(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return log_posterior::Float64, log_posterior_grad::Array{Float64,2}
end

In [ ]:
# Run stereo algorithm using gradient ascent or sth similar
function stereo(x0::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return x::Array{Float64,2}
end

In [ ]:
function problem3()
    # use problem 2's load_data
    im0, im1, gt = load_data()

    # Display stereo: Initialized with constant 8's


    # Display stereo: Initialized with noise in [0,14]


    # Display stereo: Initialized with gt


    # Coarse to fine estimation..


end
